<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkBD0211ENSkillsNetwork885-2023-01-01"><img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="400"> </a>


<h1 align = "center"> Spark Fundamentals I - Introduction to Spark</h1>
<h2 align = "center"> Getting Started</h2>
<br align = "left">

**Related free online courses:**

Related courses can be found in the following learning paths:

- [Spark Fundamentals path](http://cocl.us/Spark_Fundamentals_Path)
- [Big Data Fundamentals path](http://cocl.us/Big_Data_Fundamentals_Path) 

<img src="http://spark.apache.org/images/spark-logo.png" height="100">


 ## Spark is built around speed and the ease of use. In these labs you will see for yourself how easy it is to get started using Spark. 

Spark’s primary abstraction is a distributed collection of items called a Resilient Distributed Dataset or RDD. In a subsequent lab exercise, you will learn more about the details of RDD. RDDs have actions, which return values, and transformations, which return pointers to new RDD.

This set of labs uses Skills Network (SN) Labs to provide an interactive environment to develop applications and analyze data. It is available in either Scala or Python shells. Scala runs on the Java VM and is thus a good way to use existing Java libraries. In this lab exercise, we will set up our environment in preparation for the later labs.

After completing this set of hands-on labs, you should be able to:

1. Perform basic RDD actions and transformations
2. Use caching to speed up repeated operations


### Using this notebook

This is an interactive environment where you can show your code through cells, and documentation through markdown.

Look at the top right corner. Do you see "Python 3"? This indicates that you are running Python in this notebook.

**To run a cell:** Shift + Enter

### Try creating a new cell below.

**To create a new cell:** In the menu, go to _"Insert" > "Insert Cell Below"_. Or, click outside of a cell, and press "a" (insert cell above) or "b" (insert cell below).


# Lab Setup

Run the following cells to get the lab data.


First, we will install the [`skillsnetwork` Python library](https://pypi.org/project/skillsnetwork/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkBD0211ENSkillsNetwork885-2023-01-01) to help manage datasets:


In [6]:
%pip install -Uq "skillsnetwork>=0.20.6"
# !pip install -Uq "skillsnetwork>=0.20.6"
# !pipx install "skillsnetwork>=0.20.6" --include-deps
# NOTES: 
# - Install ipykernel directly in WSL
# - Restart Kernel after installing packages

Note: you may need to restart the kernel to use updated packages.


In [7]:
import skillsnetwork
from packaging import version

if not hasattr(skillsnetwork, "__version__") or version.parse(skillsnetwork.__version__) < version.parse("0.20.6"):
    raise ValueError("Please install skillsnetwork>=0.20.6 or this lab won't work.")

Use the [`skillsnetwork.prepare`](https://ibm-skills-network.github.io/skillsnetwork-python-library/api/generated/skillsnetwork.prepare.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkBD0211ENSkillsNetwork885-2023-01-01) function to prepare our data:


In [21]:
# Download the data from the IBM server.
# This may take ~30 seconds depending on your internet speed.
# await skillsnetwork.prepare("https://cocl.us/BD0211EN_Data", overwrite=True)
await skillsnetwork.prepare("https://cocl.us/BD0211EN_Data", overwrite=True)

  0%|          | 0/26 [00:00<?, ?it/s]

Saved to '.'


The data is in a folder called **LabData**. Let's list all the files in the data that we just downloaded and extracted.


In [22]:
from pathlib import Path
for path in Path("LabData").iterdir():
    print(path)

LabData/users.txt
LabData/README.md
LabData/nyctaxisub.csv
LabData/.DS_Store
LabData/pom.xml
LabData/followers.txt
LabData/taxistreams.py
LabData/nyctaxi.csv
LabData/nycweather.csv
LabData/notebook.log
LabData/nyctaxi100.csv


Should have:
    
* followers.txt
* notebook.log
* nyctaxi100.csv
* nyctaxi.csv
* nyctaxisub.csv
* nycweather.csv
* pom.xml
* README.md
* taxistreams.py
* users.txt


### Starting with Spark



Let's first import the tools that we need to use Spark in this SN Labs.


In [10]:
# !mamba install -qy -c conda-forge findspark pyspark
!pip install findspark pyspark


In [23]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext.getOrCreate()

The notebooks provide code assist. For example, type in "sc." followed by the Tab key to get the list of options associated with the spark context:


In [24]:
sc.appName

'pyspark-shell'

In [17]:
sc.uiWebUrl

'http://172.28.227.43:4040'

To run a command as code, simple select the cell you want to run and either:

* Click the play button in the toolbar above
* Press "_Shift+Enter_"

Let's run a basic command and check the version of Spark running:


In [25]:
sc.version

'3.5.0'

Add in the path to the *README.md* file in **LabData**.


In [26]:
readme = sc.textFile("LabData/README.md")


In [29]:
print(readme)

LabData/README.md MapPartitionsRDD[12] at textFile at NativeMethodAccessorImpl.java:0


Let’s perform some RDD actions on this text file. Count the number of items in the RDD using this command:


In [30]:
# sc.parallelize([2, 3, 4]).count()
readme.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.ipc.RpcException: RPC response has invalid length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1933)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


You should see that this RDD action returned a value of 103.

Let’s run another action. Run this command to find the first item in the RDD:


In [31]:
readme.first()

Py4JJavaError: An error occurred while calling o89.partitions.
: org.apache.hadoop.ipc.RpcException: RPC response has invalid length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1933)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


Now let’s try a transformation. Use the filter transformation to return a new RDD with a subset of the items in the file. Type in this command:


In [32]:
linesWithSpark = readme.filter(lambda line: "Spark" in line)

You can even chain together transformations and actions. To find out how many lines contains the word “Spark”, type in:


In [33]:
linesWithSpark = readme.filter(lambda line: "Spark" in line)
readme.filter(lambda line: "Spark" in line).count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.ipc.RpcException: RPC response has invalid length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1933)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


# More on RDD Operations

This section builds upon the previous section. In this section, you will see that RDD can be used for more complex computations. You will find the line from that "README.md" file with the most words in it.

Run the following cell.


In [34]:
readme.map(lambda line: len(line.split())).reduce(lambda a, b: a if (a > b) else b)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.ipc.RpcException: RPC response has invalid length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1933)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


There are two parts to this. The first maps a line to an integer value, the number of words in that line. In the second part reduce is called to find the line with the most words in it. The arguments to map and reduce are Python anonymous functions (lambdas), but you can use any top level Python functions. In the next step, you’ll define a max function to illustrate this feature.

Define the max function. You will need to type this in:


In [35]:
def my_max(a, b):
    if a > b:
        return a
    else:
        return b

Now run the following with the max function:


In [ ]:
readme.map(lambda line: len(line.split())).reduce(my_max)

Spark has a MapReduce data flow pattern. We can use this to do a word count on the readme file.


In [36]:
wordCounts = readme.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)

Py4JJavaError: An error occurred while calling o89.partitions.
: org.apache.hadoop.ipc.RpcException: RPC response has invalid length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1933)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


Here we combined the flatMap, map, and the reduceByKey functions to do a word count of each word in the readme file.

To collect the word counts, use the _collect_ action.

#### It should be noted that the collect function brings all of the data into the driver node. For a small dataset, this is acceptable but, for a large dataset this can cause an Out Of Memory error. It is recommended to use collect() for testing only. The safer approach is to use the take() function e.g. print take(n)


In [ ]:
wordCounts.collect()

### <span style="color: red">YOUR TURN:</span> 

#### In the cell below, determine what is the most frequent word in the README, and how many times was it used?


In [ ]:
# WRITE YOUR CODE BELOW




Double-click __here__ for the solution.
<!-- The correct answer is:
wordCounts.reduce(lambda a, b: a if (a[1] > b[1]) else b)
-->


## Using Spark caching

In this short section, you’ll see how Spark caching can be used to pull data sets into a cluster-wide in-memory cache. This is very useful for accessing repeated data, such as querying a small “hot” dataset or when running an iterative algorithm. Both Python and Scala use the same commands.

As a simple example, let’s mark our linesWithSpark dataset to be cached and then invoke the first count operation to tell Spark to cache it. Remember that transformation operations such as cache does not get processed until some action like count() is called. Once you run the second count() operation, you should notice a small increase in speed.


In [37]:
print(linesWithSpark.count())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.ipc.RpcException: RPC response has invalid length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1933)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


In [ ]:
from timeit import Timer
def count():
    return linesWithSpark.count()
t = Timer(lambda: count())

In [ ]:
print(t.timeit(number=50))

In [ ]:
linesWithSpark.cache()
print(t.timeit(number=50))

It may seem silly to cache such a small file, but for larger data sets across tens or hundreds of nodes, this would still work. The second linesWithSpark.count() action runs against the cache and would perform significantly better for large datasets.


<div class="alert alert-success alertsuccess" style="margin-top: 20px">
    <strong>Tip</strong>: Enjoyed using Jupyter notebooks with Spark? Get yourself a free 
    <a href="http://cocl.us/DSX_on_Cloud">IBM Cloud</a> account where you can use Data Science Experience notebooks
    and have <em>two</em> Spark executors for free!
</div>


### Summary
Having completed this exercise, you should now be able to log in to your environment and use the Spark shell to run simple actions and transformations for Scala and/or Python. You understand that Spark caching can be used to cache large datasets and subsequent operations on it will utilize the data in the cache rather than re-fetching it from HDFS.


This notebook is part of the free course on **cognitiveclass.ai** called *Spark Fundamentals I*. If you accessed this notebook outside the course, you can take this free self-paced course online by going to: http://cocl.us/Spark_Fundamentals_I


### About the Authors:  
Hi! It's Alex Aklson, one of the authors of this notebook. I hope you found this lab educational! There is much more to learn about Spark but you are well on your way. Feel free to connect with me if you have any questions.
<hr>
